In [1]:
import open3d
import argparse
import sys
sys.path.append('.')
import os
import time
import h5py
import datetime
import numpy as np
from matplotlib import pyplot as plt
import torch
import torch.nn.parallel
import torch.nn as nn
import torch.utils.data
import torch.optim as optim
from torch.utils.data import DataLoader
import torch.nn.functional as F
from data_utils.ModelNetDataLoader import ModelNetDataLoader, load_data, class_names
from pathlib import Path
from tqdm import tqdm
from utils import test, save_checkpoint, select_avaliable, mkdir
from model.pointnet2 import PointNet2ClsMsg
from model.pointnet import PointNetCls, feature_transform_reguliarzer
import colorsys
from clf import parse_args, root

import my_log as log
from my_fn import *

args = parse_args([])
args.pretrain = 'experiment/weights/clf-pointnet-0.89730-0076.pth'
args.batch_size = 8
test_data, test_label = load_data(root, train = False)
testDataset = ModelNetDataLoader(test_data, test_label)
testDataLoader = torch.utils.data.DataLoader(testDataset, batch_size=args.batch_size, shuffle=False)

log.debug('Building Model',args.model_name)
if args.model_name == 'pointnet':
    num_class = 40
    model = PointNetCls(num_class,args.feature_transform).cuda()  
else:
    model = PointNet2ClsMsg().cuda()

if args.pretrain is None:
    log.err('No pretrain model')
else:
    log.debug('Loading pretrain model...')
    checkpoint = torch.load(args.pretrain)
    model.load_state_dict(checkpoint)
    model.eval()

pretrain: experiment/weights/clf-pointnet-0.89730-0076.pth 
batch_size: 16 gpu_count: 1 
Building Model pointnet 
Loading pretrain model... 


In [2]:
def num(x):
    return x.detach().cpu().numpy()

def disp_pt_cloud(x,channal_first = True):
    cloud = open3d.geometry.PointCloud()
    cloud.points = open3d.utility.Vector3dVector(x)

    vis = open3d.visualization.VisualizerWithKeyCallback()
    vis.create_window(args.model_name, height=800, width=800, left=200, top=0)
    opt = vis.get_render_option().background_color = np.asarray([0, 0, 0])
    vis.add_geometry(cloud)
    vis.register_key_callback(32, lambda vis: exit())
    vis.run()
    vis.destroy_window()

for pts, gt in testDataLoader:
    gt = gt[:, 0].long().cuda()
    pts.requires_grad = False
    pts = pts.transpose(2, 1).cuda() # make channal first

    output, _ = model(pts)
    org_pred = output.data.max(1)[1]
    target = torch.from_numpy(np.array([0]* args.batch_size)).cuda()
    noise = torch.zeros_like(pts, requires_grad=True).cuda()
    opt = optim.Adam([noise], lr=0.05, betas=(0.9, 0.999))
    # mse_fn = nn.MSELoss()
    # loss_l2 = mse_fn(pts_adv, pts)
    crossEntropy_fn = nn.CrossEntropyLoss()
    
    for i in range(40):        
        opt.zero_grad()
        pts_adv = pts + noise
        softmax, _ = model(pts_adv)
        loss_cross = crossEntropy_fn(softmax, target)
        loss_chamfer = chamfer_batch(pts_adv.transpose(2, 1), pts.transpose(2, 1))
        loss = loss_cross*0.5 + loss_chamfer
        loss.backward()
        opt.step()
            
        output, _ = model(pts_adv)
        adv_chocie = output.data.max(1)[1]
        not_org = num((adv_chocie == org_pred).float().mean())
        is_target = num((adv_chocie == target).float().mean())
        log.info(num(loss),cross=num(loss_cross),chamfer=num(loss_chamfer),not_org=not_org, is_target=is_target)
        del output,adv_chocie,not_org,is_target
        
    break

7.1974826 cross: 14.394965 chamfer: 0.0 not_org: 1.0 is_target: 0.125 
33.48239 cross: 9.6157875 chamfer: 28.674496 not_org: 1.0 is_target: 0.125 
30.276712 cross: 3.924061 chamfer: 28.314682 not_org: 0.375 is_target: 0.125 
31.92022 cross: 2.4449725 chamfer: 30.697733 not_org: 0.375 is_target: 0.25 
34.746876 cross: 2.0026302 chamfer: 33.74556 not_org: 0.125 is_target: 0.375 
33.74937 cross: 1.4878949 chamfer: 33.005424 not_org: 0.25 is_target: 0.5 
31.757317 cross: 1.0859597 chamfer: 31.214336 not_org: 0.25 is_target: 0.625 
30.564236 cross: 0.9095247 chamfer: 30.109474 not_org: 0.375 is_target: 0.625 
30.305092 cross: 0.83326566 chamfer: 29.888458 not_org: 0.125 is_target: 0.75 
29.978903 cross: 0.7398648 chamfer: 29.60897 not_org: 0.25 is_target: 0.875 
28.83044 cross: 0.70417 chamfer: 28.478355 not_org: 0.125 is_target: 1.0 
27.439226 cross: 0.75322986 chamfer: 27.06261 not_org: 0.125 is_target: 0.875 
26.308908 cross: 0.60402274 chamfer: 26.006897 not_org: 0.125 is_target: 1.0 
2

In [8]:
# disp_pt_cloud(num(pts.detach().transpose(2, 1)[0]))
disp_pt_cloud(num(pts_adv.detach().transpose(2, 1)[0]))

In [5]:
aa, _ = model(pts)
bb, _ = model(pts_adv)
print(num(aa.data.max(1)[1]))
print(num(bb.data.max(1)[1]))

[ 4  0  2  8 23 37 35 35]
[0 0 0 0 0 0 0 0]
